In [ ]:
# All code run on AWS m4.2xlarge (8 CPUs and 32 GB of RAM) with Python 3.

In [ ]:
from multiple_cluster_engine import MultipleClusterEngine

# create some fake files
for i in range(10):
    !echo {i + 10} > {i}.tmp        
        

def save_a_string(string_saved_to_file, # actual args used in this function
    # mandatory args, you can choose not to use them but function has to take them in
     input_file_name, cluster_output_dir, cluster_id, n_cpus, cpu_id  
):
    import os # function has to import all the libraries it uses
    with open(input_file_name, 'r') as in_:
        text = ''.join(in_.readlines())        
    output_file_name = '_'.join(['cluster' + str(cluster_id), 'cpu' + str(cpu_id), input_file_name.split('/')[-1]])
    output_file_name = os.path.join(cluster_output_dir, output_file_name)
    with open(output_file_name, 'w') as out_:
        out_.write('My cluster_id is {}\n'.format(cluster_id))
        out_.write('The number of CPUs in this cluster is {}\n'.format(n_cpus))
        out_.write('My CPU_id is {}\n'.format(cpu_id))
        out_.write('My string is: {}\n'.format(string_saved_to_file))

mce_args = {
    'cluster_job_name': 'save_a_silly_string_', # no spaces as it will be part of directory name
    'n_cpus_list': [4, 3, 2], # 1st cluster is always the largest or equal to the other clusters
    'ram_limit_in_GB': 20.0,
    'wait_time_in_seconds': 1,
    'input_file_names': ['{}.tmp'.format(i) for i in range(10)], # absolute path preferred; I'm lazy
    'output_parent_dir': '/home/ubuntu/cluster_results', # absolute path preferred, directory has to already exist
    'function_to_process': save_a_string,
    'function_kwargs_dict': {'string_saved_to_file': 'pee-a-boo!'} # function arguments here
    }

mce = MultipleClusterEngine(**mce_args)
# mce.start_all_clusters()
# mce.run_clusters()
mce.main()

In [2]:
mce.async_results_dict # contains the details of each AsyncResultObject such as time to complete, etc.

defaultdict(list,
            {0: [<AsyncMapResult: <lambda>:finished>,
              <AsyncMapResult: <lambda>:finished>,
              <AsyncMapResult: <lambda>:finished>,
              <AsyncMapResult: <lambda>:finished>],
             1: [<AsyncMapResult: <lambda>:finished>,
              <AsyncMapResult: <lambda>:finished>,
              <AsyncMapResult: <lambda>:finished>],
             2: [<AsyncMapResult: <lambda>:finished>,
              <AsyncMapResult: <lambda>:finished>,
              <AsyncMapResult: <lambda>:finished>]})

In [ ]:
# mce.kill_all_clusters() # if your function_to_process failed while clusters still alive, then kill clusters manually
# probably still have to take a look at the processes and manually kill the clusters using the following lines
# to kill each of the 3 clusters
# !ipcluster stop --profile save_to_string_0
# !ipcluster stop --profile save_to_string_1
# !ipcluster stop --profile save_to_string_2

In [1]:
from multiple_cluster_engine import MultipleClusterEngine

def error_func1( # this function takes no real args that are not mandatory
    # mandatory args, you can choose not to use them but function has to take them in
     input_file_name, cluster_output_dir, cluster_id, n_cpus, cpu_id ):
    1 / 0
    
mce_args = {
    'cluster_job_name': 'error_one_', # no spaces as it will be part of directory name
    'n_cpus_list': [4, 3, 2], # 1st cluster is always the largest or equal to the other clusters
    'ram_limit_in_GB': 20.0,
    'wait_time_in_seconds': 1,
    'input_file_names': ['{}.tmp'.format(i) for i in range(10)],
    'output_parent_dir': '/home/ubuntu/cluster_results', # use absolute path since it's safer, has to already exist
    'function_to_process': error_func1,
    'function_kwargs_dict': {} # error_func1 takes no additional arguments    
    }

mce1 = MultipleClusterEngine(**mce_args)
mce1.main()

100%|██████████| 10/10 [00:10<00:00,  1.01s/it]


In [2]:
# mce1.kill_all_clusters()
mce1.async_results_dict # when a function fails on a file, its async_result history is removed

defaultdict(list, {0: [], 1: [], 2: []})

In [1]:
from multiple_cluster_engine import MultipleClusterEngine


def error_func2( # this function takes no real args that are not mandatory
    # mandatory args, you can choose not to use them but function has to take them in 
     input_file_name, cluster_output_dir, cluster_id, n_cpus, cpu_id):
    '1' + 2
    
mce_args = {
    'cluster_job_name': 'error_two_', # no spaces as it will be part of directory name
    'n_cpus_list': [4, 3, 2], # 1st cluster is always the largest or equal to the other clusters
    'ram_limit_in_GB': 20.0,
    'wait_time_in_seconds': 1,
    'input_file_names': ['{}.tmp'.format(i) for i in range(10)],
    'output_parent_dir': '/home/ubuntu/cluster_results', # use absolute path since it's safer, has to already exist
    'function_to_process': error_func2,
    'function_kwargs_dict': {} # error_func2 takes no additional arguments    
    }

mce2 = MultipleClusterEngine(**mce_args)
mce2.main()

100%|██████████| 10/10 [00:10<00:00,  1.01s/it]


In [3]:
# mce2.kill_all_clusters()
mce2.async_results_dict # when a function fails on a file, its async_result history is removed

defaultdict(list, {0: [], 1: [], 2: []})

In [1]:
from multiple_cluster_engine import MultipleClusterEngine


def exceed_memory_limit( # this function takes no real args that are not mandatory
    # mandatory args, you can choose not to use them but function has to take them in 
     input_file_name, cluster_output_dir, cluster_id, n_cpus, cpu_id):
    import time
    time.sleep(5)
    if cluster_id % 2: # if cluster_id is odd, increase RAM usage until it is killed
        exceed_ram_limit = []
        current_value = 0
        while True:
            exceed_ram_limit.append(current_value)
            current_value += 1
    return cluster_id
        
    
mce_args = {
    'cluster_job_name': 'exceed_memory_', # no spaces as it will be part of directory name
    'n_cpus_list': [4, 3, 2, 1, 1, 1], # 1st cluster is always the largest or equal to the other clusters
    'ram_limit_in_GB': 20.0,
    'wait_time_in_seconds': 1,
    'input_file_names': ['{}.tmp'.format(i) for i in range(10)],
    'output_parent_dir': '/home/ubuntu/cluster_results', # use absolute path since it's safer, has to already exist
    'function_to_process': exceed_memory_limit,
    'function_kwargs_dict': {} # exceed_memory_limit takes no additional arguments    
    }

mce3 = MultipleClusterEngine(**mce_args)
mce3.main()

100%|██████████| 10/10 [00:13<00:00,  1.31s/it]


In [ ]:
mce3.async_results_dict # when a function fails due to exceeding memory limit, its async_result history is removed